<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.3: Higher-Order Functions | 高层次函数
**Prev:上一个小节： [Interlude: Chisel Standard Library | 间歇：Chisel 标准库](3.2_interlude.ipynb)**<br>
**Next:下一个小节： [Functional Programming | 函数式编程](3.4_functional_programming.ipynb)**

## Motivation | 动机
Those pesky `for` loops in the previous module are verbose and defeat the purpose of functional programming! In this module, your generators will get funct-ky. 

在之前单元中，那些烦人的 `for` 循环是冗杂的并且也违背了函数式编程的意愿。在本单元中，你的生成器将会更加有效率。

## Setup | 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# A Tale of Two FIRs <a name="compact-fir"></a>
From the last module, we had the convolution part of the FIR filter written like this: 

在上一个单元中，我们创造了 FIR 滤波器中的卷积部分，它像下面这样写的：

```scala
val muls = Wire(Vec(length, UInt(8.W)))
for(i <- 0 until length) {
  if(i == 0) muls(i) := io.in * io.consts(i)
  else       muls(i) := regs(i - 1) * io.consts(i)
}

val scan = Wire(Vec(length, UInt(8.W)))
for(i <- 0 until length) {
  if(i == 0) scan(i) := muls(i)
  else scan(i) := muls(i) + scan(i - 1)
}

io.out := scan(length - 1)
```

As a recap, the idea is to multiply each element of `io.in` with the corresponding element of `io.consts`, and store it in `muls`. 
Then, the elements in `muls` are accumulated into `scan`, with `scan(0) = muls(0)`, `scan(1) = scan(0) + muls(1) = muls(0) + muls(1)`, and in general `scan(n) = scan(n-1) + muls(n) = muls(0) + ... + muls(n-1) + muls(n)`. 
The last element in `scan` (equal to the sum of all `muls`) is assigned to `io.out`. 
However, it's very verbose for what might be considered quite a simple operation. In fact, all that could be written in one line: 

回顾一下，我们的编程思路是把每一个 `io.in` 的元素与 `io.consts` 的元素对应相乘，然后把所得的积存入到 `muls`。然后，`muls` 中的所有元素通过`scan(0) = muls(0)`, `scan(1) = scan(0) + muls(1) = muls(0) + muls(1)`累加到 `scan` 中。也就是说，`scan(n) = scan(n-1) + muls(n) = muls(0) + ... + muls(n-1) + muls(n)`。 在 `scan` 中的最后一个元素 (等于所有 `muls` 的累加和) 被赋值到 `io.out` 中。

然而，像这样编写一个想起来很简单的操作是很冗余的。实际上，所有的语句都可以用一行表示。

```scala
io.out := (taps zip io.consts).map { case (a, b) => a * b }.reduce(_ + _)
```

What is it doing?! Let's break it down: 
- `(taps zip io.consts)` takes two lists, `taps` and `io.consts`, and combines them into one list where each element is a tuple of the elements at the inputs at the corresponding position. Concretely, its value would be `[(taps(0), io.consts(0)), (taps(1), io.consts(1)), ..., (taps(n), io.consts(n))]`. Remember that periods are optional, so this is equivalent to `(taps.zip(io.consts))`. 
- `.map { case (a, b) => a * b }` applies the anonymous function (takes a tuple of two elements returns their product) to the elements of the list, and returns the result. In this case, the result is equivalent to `muls` in the verbose example, and has the value `[taps(0) * io.consts(0), taps(1) * io.consts(1), ..., taps(n) * io.consts(n)]`. You'll revisit anonymous functions in the next module. For now, just learn this syntax.
- Finally, `.reduce(_ + _)` also applies the function (addition of elements) to elements of the list. However, it takes two arguments: the first is the current accumulation, and the second is the list element (in the first iteration, it just adds the first two elements). These are given by the two underscores in the parentheses. The result would then be, assuming left-to-right traversal, `(((muls(0) + muls(1)) + muls(2)) + ...) + muls(n)`, with the result of deeper-nested parentheses evaluated first. This is the output of the convolution. 

非常神奇有没有？这是怎么做到的？让我们分开解释。
- `(taps zip io.consts)`占用了两个列表，`taps` 和 `io.consts`，然后把它们整合到同一个列表中去；在这个新列表中，每一个元素都是两个输入元素组成的对应的元胞。也就是说，这个新列表的每一个元素值将会是 `[(taps(0), io.consts(0)), (taps(1), io.consts(1)), ..., (taps(n), io.consts(n))]`。考虑到句号是可选的，所以也可以写成这样`(taps.zip(io.consts))`。
- `.map { case (a, b) => a * b }`实现了列表元素的匿名函数 (接收一个有两个元素的元胞，返回它们的积) ，并且返回结果。在这里，返回值是先前冗长的例子，等于 `[taps(0) * io.consts(0), taps(1) * io.consts(1), ..., taps(n) * io.consts(n)]`。你在下一个单元中还会遇到匿名函数。现在，我们只是学习这个语法。
- 最后，`.reduce(_ + _)` 也实现了列表元素的功能 (元素求和) 。虽然，它有两个参数：第一个是现在的累加值，第二个是在列表中的元素 (在第一个循环中，它只把前两个元素相加) 。这是原括弧中两个下划线的含义。结果将会是，假设从左到右循环，`(((muls(0) + muls(1)) + muls(2)) + ...) + muls(n)`，更里面原括弧先计算。这就是卷积的输出值。

---
# Functions as Arguments | 函数参数
Formally, functions like `map` and `reduce` are called _higher-order functions_: they are functions that take functions as arguments. 
As it turns out (and hopefully, as you can see from the above example), these are very powerful constructs that encapsulate a general computational pattern, allowing you to concentrate on the application logic instead of flow control, and resulting in very concise code. 

`map` 和 `reduce` 这样的函数可以被比较正式地称为 _高级函数_：这些函数把低一级的函数作为参数。
就像你在上面看到的例子一样，当它们出现的时候，通常有一些有力的构造可以概括地描述计算模式，这使得你可以集中注意力在应用逻辑上而不是流程控制，并且也可以产生很简明的代码。

## Different ways of specifying functions | 定义函数的不同方法
You may have noticed that there were two ways of specifying functions in the examples above: 
- For functions where each argument is referred to exactly once, you *may* be able to use an underscore (`_`) to refer to each argument. In the example above, the `reduce` argument function took two arguments and could be specified as `_ + _`. While convenient, this is subject to an additional set of arcane rules, so if it does't work, try: 
- Specifying the inputs argument list explicitly. The reduce could have been explicitly written as `(a, b) => a + b`, with the general form of putting the argument list in parentheses, followed by `=>`, followed by the function body referring to those arguments. 
- When tuple unpacking is needed, using a `case` statement, as in `case (a, b) => a * b`. That takes a single argument, a tuple of two elements, and unpacks it into variables `a` and `b`, which can then be used in the function body. 

你可能注意到了，在上面的例子中有两种描述函数的方法：
- 在一些函数里面。每一个参数都被指定了具体的值，是*可能*可以使用下划线 (`_`) 来代指每一个参数。在上面这个例子当中，`reduce` 参数函数接收两个参数，能够特指为 `_ + _`。但是在转换的时候，会被添加一堆深奥的规则，所以这就不成功了，试试看：
- 隐式地指定输入的参数。 `reduce` 的隐式传参是 `(a, b) => a + b`，括号包裹参数列表，这是一个普通的样式；后面是`=>`，最后是指明了参数的函数体。
- 当元胞需要拆开时，使用 `case` 语句，两个元素的元胞，会被拆开为两个变量 `a` 和 `b`，这在之后就可以被函数体使用。

## Practice in Scala | 通过 Scala 练习
In the last module, we've seen major classes in the Scala Collections API, like `List`s. 
These higher-order functions are part of these APIs - and in fact, the above example uses the `map` and `reduce` API on `List`s.
In this section, we'll familiarize ourselves with these methods through examples and exercises. 
In these examples, we'll operate on Scala numbers (`Int`s) for the sake of simpliciy and clarify, but because Chisel operators behave similarly, the concepts should generalize. 

在上一个单元中，我们看到了 Scala 集合 API 中的主要类，就像 `List`。
这些等级的函数是那些 API 的一部分——实际上，上面的例子就使用了 `map` 和 `reduce` API，而这就是 `List` 的一部分。
为了简单和清晰，在这些例子中，我们只对 Scala 的数据类型 (`Int`) 进行操作，但是 Chisel 的操作是类似的，所以这些概念是通用的。

<span style="color:blue">**Example: Map | 举例：映射**</span><br>
`List[A].map` has type signature `map[B](f: (A) ⇒ B): List[B]`. You'll learn more about types in a later module. For now, think of types A and B as `Int`s or `UInt`s, meaning they could be software or hardware types.  

In plain English, it takes an argument of type `(f: (A) ⇒ B)`, or a function that takes one argument of type `A` (the same type as the element of the input List) and returns a value of type `B` (which can be anything). `map` then returns a new list of type `B` (the return type of the argument function). 

As we've already explained the behavior of List in the FIR explanation, let's get straight into the examples and exercises: 

`List[A].map` 有一个函数签名 `map[B](f: (A) ⇒ B): List[B]`。在后面的几个单元中你会学到更多。现在，只是把 A 和 B 类型看作 `Int` 或者 `UInt`，意味着它们可以是 Scala 类型或者 Chisel 类型。

意思是，接收类型为 `(f: (A) ⇒ B)` 的参数。或者说，有一个函数接收类型 `A` 作为参数 (与输入列表相同的类型)，并且返回类型 `B` (或者其他任何类型)。`map` 然后返回一个新的 `B` 类型的列表 (参数函数的返回类型)。

在 FIR 例子中我们已经解释了列表的行为，让我们直接来看例子和练习吧：

In [ ]:
println(List(1, 2, 3, 4).map(x => x + 1))  
// explicit argument list in function 
// 在函数中使用显式参数列表
println(List(1, 2, 3, 4).map(_ + 1))  
// equivalent to the above, but implicit arguments 
// 与上面相同，只不过使用隐式参数
println(List(1, 2, 3, 4).map(_.toString + "a"))  
// the output element type can be different from the input element type 
// 输出元素的类型可以与输入元素的类型不同。

println(List((1, 5), (2, 6), (3, 7), (4, 8)).map { case (x, y) => x*y })  
// this unpacks a tuple, note use of curly braces 
// 这拆开了元胞，注意使用的尖括号。

// Related: Scala has a syntax for constructing lists of sequential numbers 
// 相似的：Scala 也有一个构造序列数字的列表的语法
println(0 to 10)  
// to is inclusive , the end point is part of the result 
// to 代表包括，结束点也是结果的一部分
println(0 until 10)  
// until is exclusive at the end, the end point is not part of the result 
// until 是不包括结尾，结束点不是结果的一部分

// Those largely behave like lists, and can be useful for generating indices:  
// 这些行为上很像列表，也可以用来产生索引：
val myList = List("a", "b", "c", "d")
println((0 until 4).map(myList(_)))

<span style="color:red">**Exercise: Map | 练习：映射**</span><br><a name="map-exercise"></a>

In [ ]:
// Now you try: 
// Fill in the blanks (the ???) such that this doubles the elements of the input list. 
// This should return: List(2, 4, 6, 8) 
// 现在你尝试：
// 补全问号处的代码，使输入列表中的每一个元素都乘上二
// 这应该返回 List(2, 4, 6, 8)
println(List(1, 2, 3, 4).map(???))

<span style="color:blue">**Example: zipWithIndex | 举例：zipWithIndex**</span><br>

`List.zipWithIndex` 有一个函数签名 `zipWithIndex: List[(A, Int)]`。
这不需要参数，但是返回一个列表。在这个列表中，每个元素都是原有元素和它索引 (从零开始) 的元组。
所以 `List("a", "b", "c", "d").zipWithIndex` 将会返回 `List(("a", 0), ("b", 1), ("c", 2), ("d", 3))`

在有些操作中会需要索引，那么这就很有用了。

既然这是显而易见的概念，我们就直接看看例子吧：

In [ ]:
println(List(1, 2, 3, 4).zipWithIndex)  // note indices start at zero 注意索引从零开始
println(List("a", "b", "c", "d").zipWithIndex)
println(List(("a", "b"), ("c", "d"), ("e", "f"), ("g", "h")).zipWithIndex)  // tuples nest 

<span style="color:blue">**Example: Reduce 举例：Reduce**</span><br>
`List[A].map` has type signature similar to `reduce(op: (A, A) ⇒ A): A`. (it's actually more lenient, `A` only has to be a supertype of the List type, but we're not going to deal with that syntax here)

As it's also been explained above, here are some examples: 

`List[A].map`的函数签名与 `reduce(op: (A, A) ⇒ A): A` 很像。实际上类型更加多样，`A` 只需要是列表的一个超级类型，但是我们不在这里深入讨论这个语法。

既然之前我们已经解释过了，那就来看些例子吧：

In [ ]:
println(List(1, 2, 3, 4).reduce((a, b) => a + b))  // returns the sum of all the elements 返回所有元素的和
println(List(1, 2, 3, 4).reduce(_ * _))  // returns the product of all the elements 返回所有元素的积
println(List(1, 2, 3, 4).map(_ + 1).reduce(_ + _))  // you can chain reduce onto the result of a map 你可以把映射的结果与 reduce 给连起来

In [ ]:
// Important note: reduce will fail with an empty list 请注意：如果是空列表，reduce 将会失败
println(List[Int]().reduce(_ * _))

<span style="color:red">**Exercise: Reduce | 练习：Reduce**</span><br><a name="reduce-exercise"></a>

In [ ]:
// Now you try: 现在你尝试：
// Fill in the blanks (the ???) such that this returns the product of the double of the elements of the input list. 在 ??? 处补全代码，使得程序返回输入列表中每个元素两倍值的累乘结果。
// 应该返回： (1*2)*(2*2)*(3*2)*(4*2) = 384
println(List(1, 2, 3, 4).map(???).reduce(???))

<span style="color:blue">**Example: Fold | 举例：Flod**</span><br>
`List[A].fold` is very similar to reduce, except that you can specify the initial accumulation value. 
It has type signature similar to `fold(z: A)(op: (A, A) ⇒ A): A`. (like `reduce`, the type of `A` is also more lenient)

Notably, it takes two argument lists, the first (`z`) is the initial value, and the second is the accumulation function. 
Unlike `reduce`, it will not fail with an empty list, instead returning the initial value directly. 

Here's some examples: 

`List[A].fold` 和 `reduce` 很像，不过 `fold` 可以设定累加值的初始值。
`List[A].fold` 与 `fold(z: A)(op: (A, A) ⇒ A): A` 的函数签名很像，就像`reduce`，`A` 的类型更加宽松。
很明显，`fold` 也有两个参数列表，第一个 (`z`) 是初始值，第二个是要重复的函数。
不像 `reduce`，`fold` 在输入空列表的时候不会报错，而是直接返回初始值。

这里是一些例子：

In [ ]:
println(List(1, 2, 3, 4).fold(0)(_ + _))  // equivalent to the sum using reduce 等于和，与使用 reduce 效果一样
println(List(1, 2, 3, 4).fold(1)(_ + _))  // like above, but accumulation starts at 1 与上面相似，不过累加从1开始。
println(List().fold(1)(_ + _))  // unlike reduce, does not fail on an empty input 与 reduce 不同，在空输入的时候不会失败

<span style="color:red">**Exercise: Fold | 练习：Fold**</span><br><a name="fold-exercise"></a>

In [ ]:
// Now you try: 现在你尝试：
// Fill in the blanks (the ???) such that this returns the double the product of the elements of the input list. 在 ??? 处补全代码，使得程序返回输入列表中每个元素累乘结果的两倍。
// This should return: 2*(1*2*3*4) = 48 应该返回：2*(1*2*3*4) = 48
// Note: unless empty list tolerance is needed, reduce is a much better fit here.注意：除非需要考虑空列表的情况，不然在这里使用 reduce 更加合适
println(List(1, 2, 3, 4).fold(???)(???))

<span style="color:red">**Exercise: Decoupled Arbiter | 练习：解耦仲裁器**</span><br>
Let's put everything together now into an exercise. 

For this example, we're going to build a Decoupled arbiter: a module that has _n_ Decoupled inputs and one Decoupled output. 
The arbiter selects the lowest channel that is valid and forwards it to the output. 

Some hints: 
- Architecturally: 
  - `io.out.valid` is true if any of the inputs are valid   
  - Consider having an internal wire of the selected channel 
  - Each input's `ready` is true if the output is ready, AND that channel is selected (this does combinationally couple ready and valid, but we'll ignore it for now...) 
- These constructs may help: 
  - `map`, especially for returning a Vec of sub-elements, for example `io.in.map(_.valid)` returns a list of valid signals of the input Bundles
  - `PriorityMux(List[Bits, Bool])`, which takes in a list of bits and valid signals, returning the first element that is valid
  - Dynamic index on a Vec, by indexing with a UInt, for example `io.in(0.U)` 
  
现在我们把所有知识放在一起来做一个练习。

在这个例子中，我们要构建一个解耦仲裁器：一个有 _n_ 无关的输入和一个解耦的输出模块。
仲裁器选择最低位的有效信号，并且把这个信号输出。

一些提示：
- 架构层面：
    - 如果有任何一个输入有效，那么 `io.out.valid` 将会是高电平
    - 考虑增加一个中间信号，这个信号将记录被选中的信号位。
    - 如果输出信号是就绪的话，那么每一个输入的 `ready` 都是高电平，于是 channel 的值也通过这样的方法得到 (这确实会把就绪信号和有效信号结合起来，但是我们现在先忽视它。) 
- 这些构造体可能会有帮助：
    - `map`，特别是返回一个向量的下级元素，比如 `io.in.map(_.valid)` 返回输入 Bundles 的有效信号列表。
    - `PriorityMux(List[Bits, Bool])`，输入一个 bit 类型的列表和一个有效信号，如果有效信号是高电平，那么就返回第一个元素。
    - 在向量中动态地索引，可以使用无符号整数。比如`io.in(0.U)`。


In [ ]:
class MyRoutingArbiter(numChannels: Int) extends Module {
  val io = IO(new Bundle {
    val in = Vec(numChannels, Flipped(Decoupled(UInt(8.W))))
    val out = Decoupled(UInt(8.W))
  } )

  // Your code here 在下面补全代码
  ???
}

// verify that the computation is correct 检查计算结果是否正确
class MyRoutingArbiterTester(c: MyRoutingArbiter) extends PeekPokeTester(c) {
  // Set input defaults
  for(i <- 0 until 4) {
    poke(c.io.in(i).valid, 0)
    poke(c.io.in(i).bits, i)
    poke(c.io.out.ready, 1)
  }
    
  expect(c.io.out.valid, 0)
    
  // Check single input valid behavior with backpressure 检查单输入有效
  for (i <- 0 until 4) {
    poke(c.io.in(i).valid, 1)
    expect(c.io.out.valid, 1)
    expect(c.io.out.bits, i)
      
    poke(c.io.out.ready, 0)
    expect(c.io.in(i).ready, 0)
      
    poke(c.io.out.ready, 1)
    poke(c.io.in(i).valid, 0)
  }
    
  // Basic check of multiple input ready behavior with backpressure 多个输入信号的基础检查，带有背压的有效-就绪信号。
  poke(c.io.in(1).valid, 1)
  poke(c.io.in(2).valid, 1)
  expect(c.io.out.bits, 1)
  expect(c.io.in(1).ready, 1)
  expect(c.io.in(0).ready, 0)
    
  poke(c.io.out.ready, 0)
  expect(c.io.in(1).ready, 0)
}

val works = Driver(() => new MyRoutingArbiter(4)) {
  c => new MyRoutingArbiterTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error Scala代码，如果 works == false，那么将会报错
println("成功！！") // Scala Code: if we get here, our tests passed! Scala 代码，如果我们运行到了这里，那么我们的验证就通过啦！

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution | 解答</strong></label>
<article>
<pre style="background-color:#f7f7f7">
class MyRoutingArbiter(numChannels: Int) extends Module {
  val io = IO(new Bundle {
    val in = Vec(Flipped(Decoupled(UInt(8.W))), numChannels)
    val out = Decoupled(UInt(8.W))
  } )

  // YOUR CODE BELOW 下面是答案
  io.out.valid := io.in.map(\_.valid).reduce(\_ || \_)
  val channel = PriorityMux(
    io.in.map(\_.valid).zipWithIndex.map { case (valid, index) => (valid, index.U) }
  )
  io.out.bits := io.in(channel).bits
  for ((ready, index) <- io.in.map(\_.ready).zipWithIndex) {
    ready := io.out.ready && channel === index.U
  }
}
</pre></article></div></section></div>

---
# You're done! | 恭喜你，完成了本节内容的学习！

[Return to the top. | 回到顶层。](#top)